# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [1]:
from google.colab import userdata
userdata.get('HF_TOKEN')

ModuleNotFoundError: No module named 'google.colab'

In [28]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-guqlzug8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-guqlzug8
  Resolved https://github.com/huggingface/accelerate.git to commit 61bcdaa45d35a77478decc78ce204b54dc108812
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ejcpro8x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ejcpro8x
  Resolved https://github.com/huggingface/transformers.git to commit f065e402fc939758eabcfe182f34543258ac94ca
  Installing build dependencies ... done
  Getting requirements to build whe

KeyboardInterrupt: 

In [2]:
%pip install -q -U "transformers<5,>=4.41" "accelerate>=0.33.0" "peft>=0.11.0" bitsandbytes sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 14.7 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [8]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [11]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    - customers(customer_id, full_name, email, country, created_at)
    - orders(order_id, customer_id, order_date, status, total_amount)
    - order_items(order_item_id, order_id, sku, item_name, quantity, unit_price)


    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [12]:
sp_nl2sql = sp_nl2sql.format(question="list the orders per customer?")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    - customers(customer_id, full_name, email, country, created_at)
    - orders(order_id, customer_id, order_date, status, total_amount)
    - order_items(order_item_id, order_id, sku, item_name, quantity, unit_price)


    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `list the orders per customer?`:
    ```sql3
    


In [13]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

KeyboardInterrupt: 

In [25]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [26]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

NameError: name 'SQL' is not defined

The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [ ]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    - customers(customer_id, full_name, email, country, created_at)
    - orders(order_id, customer_id, order_date, status, total_amount)
    - order_items(order_item_id, order_id, sku, item_name, quantity, unit_price)

    ### Response
        Sample A. Orders in an inclusive id range
              Question: Which orders have ids between 29540 and 29560
              SQL:
              ```sql
              SELECT "order_id", "account_id"
              FROM "order"
              WHERE "order_id" BETWEEN 29540 AND 29560;
    Sample B. Top three authors by sold quantity
              Question: Who are the top three authors by total sold copies
              SQL:
              SELECT a."au_id", a."au_fname", a."au_lname", SUM(s."qty") AS "total_sold"
              FROM "authors" a
              JOIN "titleauthor" ta ON ta."au_id" = a."au_id"
              JOIN "titles" t ON t."title_id" = ta."title_id"
              JOIN "sales" s ON s."title_id" = t."title_id"
              GROUP BY a."au_id", a."au_fname", a."au_lname"
              ORDER BY "total_sold" DESC
              LIMIT 3;

    `{question}`:
    ```sql3
    """


In [ ]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return the  top 3 items with the large amount of orders")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    - customers(customer_id, full_name, email, country, created_at)
    - orders(order_id, customer_id, order_date, status, total_amount)
    - order_items(order_item_id, order_id, sku, item_name, quantity, unit_price)

    ### Response
        Sample A. Orders in an inclusive id range
              Question: Which orders have ids between 29540 and 29560
              SQL:
              ```sql
              SELECT "order_id", "account_id"
              FROM "order"
              WHERE "order_

In [ ]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [16]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

NameError: name 'SQL' is not defined

The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [14]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    - customers(customer_id, full_name, email, country, created_at)
    - orders(order_id, customer_id, order_date, status, total_amount)
    - order_items(order_item_id, order_id, sku, item_name, quantity, unit_price)

    ### Response
        Sample A. Orders in an inclusive id range
              Question: Which orders have ids between 29540 and 29560
              SQL:
              ```sql
              SELECT "order_id", "account_id"
              FROM "order"
              WHERE "order_id" BETWEEN 29540 AND 29560;
        Sample B. Top three authors by sold quantity
              Question: Who are the top three authors by total sold copies
              SQL:
              SELECT a."au_id", a."au_fname", a."au_lname", SUM(s."qty") AS "total_sold"
              FROM "authors" a
              JOIN "titleauthor" ta ON ta."au_id" = a."au_id"
              JOIN "titles" t ON t."title_id" = ta."title_id"
              JOIN "sales" s ON s."title_id" = t."title_id"
              GROUP BY a."au_id", a."au_fname", a."au_lname"
              ORDER BY "total_sold" DESC
              LIMIT 3;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [18]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    - customers(customer_id, full_name, email, country, created_at)
    - orders(order_id, customer_id, order_date, status, total_amount)
    - order_items(order_item_id, order_id, sku, item_name, quantity, unit_price)

    ### Response
        Sample A. Orders in an inclusive id range
              Question: Which orders have ids between 29540 and 29560
              SQL:
              ```sql
              SELECT "order_id", "account_id"
              FROM "order"
              WHERE "order_i

In [19]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

NameError: name 'tokenizer' is not defined

In [20]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

NameError: name 'SQL' is not defined

#Now the question in spanish.


In [ ]:
sp_nl2sql3 = sp_nl2sql3b.format(question="YOUR QUERY HERE")
print (sp_nl2sql3)

In [ ]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

NameError: name 'tokenizer' is not defined

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT MAX(;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [15]:
sp_nl2sql3_bank = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    customers(customer_id, full_name, email, phone, created_at)
    accounts(account_id, customer_id, account_type, currency, opened_at, status)
    transactions(tx_id, account_id, tx_type, amount, tx_date, merchant, description)

    ### Response
        Sample A. Transactions above a threshold in a recent window
              Question: Which transactions over 5000 occurred in the last 30 days
              SQL:
              ```sql
              SELECT t.tx_id, t.account_id, t.amount, t.tx_date
              FROM transactions t
              WHERE t.amount > 5000
                AND t.tx_date >= DATE('now', '-30 day');
              ```

        Sample B. Total deposits by customer
              Question: What is the total deposited amount per customer
              SQL:
              ```sql
              SELECT c.customer_id, c.full_name, SUM(t.amount) AS total_deposits
              FROM customers c
              JOIN accounts a ON a.customer_id = c.customer_id
              JOIN transactions t ON t.account_id = a.account_id
              WHERE t.tx_type = 'deposit'
              GROUP BY c.customer_id, c.full_name
              ORDER BY total_deposits DESC;
              ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
"""

In [16]:
sp_nl2sql3_bank = sp_nl2sql3_bank.format(question="Which customers made transactions over 5000 USD in the last 30 days, and what type of transaction was it?")
print (sp_nl2sql3_bank)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    customers(customer_id, full_name, email, phone, created_at)
    accounts(account_id, customer_id, account_type, currency, opened_at, status)
    transactions(tx_id, account_id, tx_type, amount, tx_date, merchant, description)

    ### Response
        Sample A. Transactions above a threshold in a recent window
              Question: Which transactions over 5000 occurred in the last 30 days
              SQL:
              ```sql
              SELECT t.tx_id, t.account_id, t.amount, t.tx_d

In [17]:
input_sentences = tokenizer(sp_nl2sql3_bank, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [18]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.customer_id, c.full_name, a.account_type, t.tx_type, t.amount, t.tx_date
    FROM customers c
    JOIN accounts a ON a.customer_id = c.customer_id
    JOIN transactions t ON t.account_id = a.account_id
    WHERE t.amount > 5000
    AND t.tx_date >= DATE('now', '-30 day')
    ORDER BY c.customer_id, a.account_type, t.tx_type, t.amount, t.tx_date;


In [ ]:
sp_nl2sql3b_fanclub = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    members(member_id, full_name, country, membership_level, joined_at)
    events(event_id, event_name, event_date, city, capacity)
    donations(donation_id, member_id, amount, donation_date, campaign)

    ### Response
        Sample A. Donors in a date range
              Question: Which members donated in the last 90 days
              SQL:
              ```sql
              SELECT DISTINCT m.member_id, m.full_name
              FROM members m
              JOIN donations d ON d.member_id = m.member_id
              WHERE d.donation_date >= DATE('now', '-90 day');
              ```

        Sample B. Top donors by total amount
              Question: Who are the top five donors by total donations
              SQL:
              ```sql
              SELECT m.member_id, m.full_name, SUM(d.amount) AS total_donated
              FROM members m
              JOIN donations d ON d.member_id = m.member_id
              GROUP BY m.member_id, m.full_name
              ORDER BY total_donated DESC
              LIMIT 5;
              ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
"""

In [ ]:
sp_nl2sql3b_fanclub = sp_nl2sql3b_fanclub.format(question="Which customers made transactions over 5000 USD in the last 30 days, and what type of transaction was it?")
print (ssp_nl2sql3b_fanclub)

In [ ]:
input_sentences = tokenizer(sp_nl2sql3b_fanclub, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

In [ ]:
sp_nl2sql3b_store = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Tables:
    customers(customer_id, name, address)
    orders(order_id, customer_id, order_date, status, total_amount)
    order_items(order_item_id, order_id, sku, item_name, quantity, unit_price)

    ### Response
        Sample A. Orders in an inclusive id range
              Question: Which orders have ids between 29540 and 29560
              SQL:
              ```sql
              SELECT order_id, customer_id
              FROM orders
              WHERE order_id BETWEEN 29540 AND 29560;
              ```

        Sample B. Top items by revenue
              Question: What are the top three items by total revenue
              SQL:
              ```sql
              SELECT oi.item_name,
                     SUM(oi.quantity * oi.unit_price) AS revenue
              FROM order_items oi
              GROUP BY oi.item_name
              ORDER BY revenue DESC
              LIMIT 3;
              ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
"""

In [ ]:
sp_nl2sql3b_store = sp_nl2sql3b_store.format(question="Which customers made transactions over 5000 USD in the last 30 days, and what type of transaction was it?")
print (sp_nl2sql3b_store)

In [ ]:
input_sentences = tokenizer(sp_nl2sql3b_store, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")